# **Cluster analysis**

### Get data from github:

In [ ]:
!wget https://raw.githubusercontent.com/shitkov/cluster_analysis/master/rutweetcorp.csv

--2021-11-23 09:30:52--  https://raw.githubusercontent.com/shitkov/cluster_analysis/master/rutweetcorp.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 519871 (508K) [text/plain]
Saving to: ‘rutweetcorp.csv’

rutweetcorp.csv     100%[===================>] 507.69K  --.-KB/s    in 0.009s  

2021-11-23 09:30:52 (55.0 MB/s) - ‘rutweetcorp.csv’ saved [519871/519871]



### Explore dataset

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('/content/rutweetcorp.csv')

In [ ]:
data.columns

Index(['text', 'label'], dtype='object')

In [ ]:
sentences = list(set(list(data['text'])))

In [ ]:
len(sentences)

2799

### Get embeddings

In [ ]:
!pip install transformers sentencepiece

     |████████████████████████████████| 3.1 MB 28.9 MB/s 
     |████████████████████████████████| 1.2 MB 21.2 MB/s 
     |████████████████████████████████| 895 kB 34.2 MB/s 
     |████████████████████████████████| 59 kB 4.2 MB/s 
     |████████████████████████████████| 596 kB 31.9 MB/s 
     |████████████████████████████████| 3.3 MB 41.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import torch

In [ ]:
from transformers import AutoTokenizer, AutoModel

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/LaBSE-en-ru")

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/806 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/509k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
model = AutoModel.from_pretrained("cointegrated/LaBSE-en-ru")

Downloading:   0%|          | 0.00/492M [00:00<?, ?B/s]

Some weights of the model checkpoint at cointegrated/LaBSE-en-ru were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
import numpy as np

In [ ]:
embeddings_list = []

for s in sentences:
    encoded_input = tokenizer(s, padding=True, truncation=True, max_length=64, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input)
    embedding = model_output.pooler_output
    embeddings_list.append((embedding)[0].numpy())

embeddings = np.asarray(embeddings_list)

### Get optimum number of clusters

In [ ]:
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity

def determine_k(embeddings):
    k_min = 2
    
    clusters = [x for x in range(2, k_min * 11)]
   
    metrics = []
    
    for i in clusters:
        metrics.append((KMeans(n_clusters=i).fit(embeddings)).inertia_)
    
    k = elbow(k_min, clusters, metrics)
    return k

def elbow(k_min, clusters, metrics):
    score = []

    for i in range(k_min, clusters[-3]):
        y1 = np.array(metrics)[:i + 1]
        y2 = np.array(metrics)[i:]
    
        df1 = pd.DataFrame({'x': clusters[:i + 1], 'y': y1})
        df2 = pd.DataFrame({'x': clusters[i:], 'y': y2})
    
        reg1 = LinearRegression().fit(np.asarray(df1.x).reshape(-1, 1), df1.y)
        reg2 = LinearRegression().fit(np.asarray(df2.x).reshape(-1, 1), df2.y)

        y1_pred = reg1.predict(np.asarray(df1.x).reshape(-1, 1))
        y2_pred = reg2.predict(np.asarray(df2.x).reshape(-1, 1))    
        
        score.append(mean_squared_error(y1, y1_pred) + mean_squared_error(y2, y2_pred))

    return np.argmin(score) + k_min

In [ ]:
k_opt = determine_k(embeddings)

In [ ]:
kmeans = KMeans(n_clusters = k_opt, random_state = 42).fit(embeddings)
kmeans_labels = kmeans.labels_

In [ ]:
data = pd.DataFrame()
data['text'] = sentences
data['label'] = kmeans_labels
data['embedding'] = list(embeddings)

In [ ]:
# Get nearest sentences to each center sentenses
from sklearn.metrics.pairwise import euclidean_distances

kmeans_centers = kmeans.cluster_centers_
top_texts_list = []
for i in range (0, k_opt):
    cluster = data[data['label'] == i]
    embeddings = list(cluster['embedding'])
    texts = list(cluster['text'])
    distances = [euclidean_distances(kmeans_centers[0].reshape(1, -1), e.reshape(1, -1))[0][0] for e in embeddings]
    scores = list(zip(texts, distances))
    top_3 = sorted(scores, key=lambda x: x[1])[:3]
    top_texts = list(zip(*top_3))[0]
    top_texts_list.append(top_texts)

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
MODEL_NAME = 'cointegrated/rut5-base-absum'
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

Downloading:   0%|          | 0.00/753 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/932M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/808k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/315 [00:00<?, ?B/s]

In [ ]:
def summarize(
    text, n_words=None, compression=None,
    max_length=1000, num_beams=3, do_sample=False, repetition_penalty=10.0, 
    **kwargs
):
    """
    Summarize the text
    The following parameters are mutually exclusive:
    - n_words (int) is an approximate number of words to generate.
    - compression (float) is an approximate length ratio of summary and original text.
    """
    if n_words:
        text = '[{}] '.format(n_words) + text
    elif compression:
        text = '[{0:.1g}] '.format(compression) + text
    # x = tokenizer(text, return_tensors='pt', padding=True).to(model.device)
    x = tokenizer(text, return_tensors='pt', padding=True)
    with torch.inference_mode():
        out = model.generate(
            **x, 
            max_length=max_length, num_beams=num_beams, 
            do_sample=do_sample, repetition_penalty=repetition_penalty, 
            **kwargs
        )
    return tokenizer.decode(out[0], skip_special_tokens=True)

In [ ]:
summ_list = []
for top in top_texts_list:
    summ_list.append(summarize(' '.join(list(top))))

In [ ]:
top_texts_list

[('я сражена наповал даже не знаю чем и парировать у меня отл за курсач по мебадури всё',
  'еще успеем пообщаться я никуда пока еще не исчезаю я так и думала что чаня это беспроигрышно спасибо за сны',
  'окай я сейчас вообще фф пишу рассказик опять там дождь по моему у меня на него манияхд'),
 ('дааа без тебя ябнесправлся спасибо тебе огрооомное с добре тебе и вот милого джерарда з ятвойдолжник',
  'шве и зоро одновременно мне выкладывают про свою любовь да личную жизнь а что я я рад',
  'как я могла пропустить зима уходи это самое долгожданное для меня на было я расстроена'),
 ('ура сегодня встречусь с колей скорей бы освободиться эх но я только вишняки проезжаю',
  'ййй перший раз я забрала мар янову р ч додому але чу мо серце шо мен погано завтра буде',
  'весь день смотрю не лоба нереально оторваться вообще шикарный в отличие от галустяна в котором вообще разочаровалась'),
 ('да детка вот онон даадададдсукаблять буду самфм крутым на райооне такое толькоу меня и у руди аха идикомн

In [ ]:
summ_list

['Спасибо за курсач по мебадури.',
 'Спасибо, что вы мне выкладывали про свою любовь да личную жизнь.',
 'Сегодня в Вишняках проезжаю с колей, чтобы освободиться от галустяна.',
 'Вот онон даадададдсукаблять буду самфм крутым на райооне',
 'Если вы чувствуете себя неважно, то слушайте.']